In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [7]:
# Импорт классики
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


$$ softmax(x)_j=\frac{exp(x_j)}{\Sigma_i exp(x_i)} $$
<br>
$$ H_t(y) = - \Sigma_i t_i \log y_i $$

In [ ]:
# Заглушка, которуб мы заполним, когда TensorFlow будет вычислять
x = tf.placeholder(tf.float32, [None, 784])
y_pred = tf.placeholder(tf.float32, [None, 10])

# Переменные весов и смещений
w = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# Модель собственной персоной
y = tf.nn.softmax(tf.matmul(x, w) + b)
# Функция ошибки - cross-entropy 
#(сначала вычисляем логарифм каждого элемента, затем y_pred 
# умножаем на соотвествующий ему логарифм, и затем суммираем результат 
# по второму измерению)
cross_entropy = tf.reduce_mean(- tf.reduce_sum(y_pred  * tf.log(y), reduction_indices=[1]))

# Градиентный спуск с learn_rate = 0.5
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# Запуск сессии в интерактивном режиме и инициализация пременных
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_pred: batch_ys})
    pass

# Получаем на выходе список булевых значений
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_pred, 1))
# Тест
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy: %s" % sess.run(accuracy, feed_dict={x: mnist.test.images, y_pred: mnist.test.labels}))

Усовершенствованная модель, со скрытым слоем и dropout

In [8]:
x = tf.placeholder(tf.float32, [None, 784])
y_pred = tf.placeholder(tf.float32, [None, 10])

# Инициализируем переменные небольшими рандомными значениями
w_relu = tf.Variable(tf.truncated_normal([784, 100], stddev=0.1))
b_relu = tf.Variable(tf.truncated_normal([100], stddev=0.1))

# Скрытый слой в общем виде (ReLU)
h = tf.nn.relu(tf.matmul(x, w_relu) + b_relu)

# Dropout
keep_probability = tf.placeholder(tf.float32)
h_drop = tf.nn.dropout(h, keep_probability)

# Третий слой (выход)
w = tf.Variable(tf.zeros([100, 10]))
b = tf.Variable(tf.zeros([10]))
# Softmax с небольшим улучшением, что бы не получить ноль в знаменателе
# Избавимся от деления на ноль 
y = tf.nn.softmax(tf.matmul(h_drop, w) + b - tf.reduce_max(x))


logit = tf.matmul(h_drop, w) + b 
# Разработчики молодцы, лучше применить эту функцию что бы избежать деления на 0
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logit, labels=y_pred))

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

for i in range(10000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_pred: batch_ys, keep_probability: 0.5})

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_pred, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy: %s" % sess.run(accuracy, feed_dict={x: mnist.test.images, y_pred: mnist.test.labels, keep_probability: 1.}))

Accuracy: 0.9750001
